In [74]:
from google.colab import drive
import pandas as pd
import numpy as np

# Acceder a Google Drive
drive.mount('/content/gdrive')
print("")

# Gráfica Direccionada
class Directed_Graph:
  def __init__(self,nodos,arcos):
    '''Se crean los nodos y los arcos'''
    self.nodos = nodos
    self.arcos = arcos

  def len_arcos(self):
    '''Obtener el numero de arcos'''
    return len(self.arcos)

  def len_nodos(self):
    '''Obtener el numero de nodos'''
    return len(self.nodos)

  def antecesores(self,nodo):
    '''Obtener los antecesores de un nodo dado'''
    if nodo not in self.nodos:
      print(nodo," no pertenece al conjunto de nodos")
      return []
    else:
      Ant = []
      for x in self.arcos:
        if nodo == x[-1]:
          Ant.append(x[0])
      return Ant

  def sucesores(self,nodo):
    '''Obtener los sucesores de un nodo dado'''
    if nodo not in self.nodos:
      print(nodo," no pertenece al conjunto de nodos")
      return []
    else:
      Ant = []
      for x in self.arcos:
        if nodo == x[0]:
          Ant.append(x[-1])
      return Ant

  def add_vertex(self, nodo):
    '''Añadir un nodo'''
    self.nodos.append(nodo)

  def add_arc(self, arco):
    '''Añadir un arco'''
    if type(arco) == tuple and len(arco) == 2:
      if arco[0] in self.nodos and arco[1] in self.nodos:
        self.arcos.append(arco)
      else:
        print("Las entradas no pertenecen a los nodos.")
    else:
      print("El arco no es valido.")

# Se recibe el archivo Excel a analizar
AE = pd.read_excel(r'/content/gdrive/My Drive/Creacion_de_la_carrera_de_mat_ap.xlsx')
Actividades = list(AE["Actividad"]) #Obtener las actividades
Duracion = list(AE["Duracion"]) #Obtener las duraciones de las actividades
Precedentes = []
for index, row in AE.iterrows(): #Asignar las actividades a sus precedentes
  actividad = row['Actividad'] #Analizar una actividad
  #Se obtienen distintas cadenas de texto segun la columna de Precedentes.
  #Si hay dos o mas precedentes, separarlos segun la coma
  lista_Precedentes = str(row['Precedentes']).split(', ')
  for precedente in lista_Precedentes: #Analizar cada uno de los precedentes
    if precedente != 'nan': #Si tiene un precedente
      precedente = int(precedente) #Cambiar el texto a un numero entero
      #Añadir a la lista una dupla donde este una actividad con su respectivo
      #precedente
      Precedentes.append((precedente, actividad))

def main():
  G = Directed_Graph(Actividades,Precedentes) #Construir la grafica
  dic = {}
  for index, row in AE.iterrows(): #Obtener nuestras actividades y precedentes
    actividad = row['Actividad'] #Obtener todas las actividades
    #Obtener todos los precedentes
    grupo_Precedentes = str(row['Precedentes']).split(', ')
    for precedente in grupo_Precedentes:
      if precedente != 'nan':
        precedente = int(precedente)
        if actividad in dic:
          dic[actividad].append(precedente)
        else:
          dic[actividad] = [precedente]
    if pd.isnull(row['Precedentes']):
      dic[actividad] = []
    #El for anterior va construyendo el diccionario

  sub = {}
  arcs = {}
  for a, ps in dic.items():
    sub[a] = []
    for p in ps:
      if p in sub:
        sub[p].append(a)
      else:
        sub[p] = [a]

  def calcular_actividades_criticas(grafo, duraciones):
    # Inicializar las duraciones más tempranas y más tardías
    duracion_mas_temprana = {nodo: 0 for nodo in grafo.nodos}
    duracion_mas_tardia = {nodo: float('inf') for nodo in grafo.nodos}

    # Calcular las duraciones más tempranas
    for nodo in grafo.nodos:
      antecesores = grafo.antecesores(nodo)
      if not antecesores:  # Si no tiene antecesores, es el nodo inicial
        duracion_mas_temprana[nodo] = 0
      else:
        duracion_mas_temprana[nodo] = max([duracion_mas_temprana[ant] + duraciones[(ant, nodo)] for ant in antecesores])

    # Calcular las duraciones más tardías
    nodos_ordenados = list(grafo.nodos)
    nodos_ordenados.sort(key=lambda x: duracion_mas_temprana[x], reverse=True)  # Ordenar nodos por duración más temprana
    for nodo in nodos_ordenados:
      sucesores = grafo.sucesores(nodo)
      if not sucesores:  # Si no tiene sucesores, es un nodo final
        duracion_mas_tardia[nodo] = duracion_mas_temprana[nodo]
      else:
        duracion_mas_tardia[nodo] = min([duracion_mas_tardia[suc] - duraciones[(nodo, suc)] for suc in sucesores])

    # Encontrar actividades críticas y sumar sus duraciones
    actividades_criticas = []
    suma_duraciones_criticas = 0
    for arco in grafo.arcos:
      duracion_arco = duraciones[arco]
      if duracion_mas_temprana[arco[0]] + duracion_arco == duracion_mas_tardia[arco[1]]:
        actividades_criticas.append(arco)
        suma_duraciones_criticas += duracion_arco

    return actividades_criticas, suma_duraciones_criticas

  # Crear el grafo y las duraciones
  grafo = Directed_Graph(Actividades, Precedentes)
  duracion_actividades = {arco: Duracion[Actividades.index(arco[1])] for arco in Precedentes}

  # Calcular y mostrar las actividades críticas y la suma de sus duraciones
  actividades_criticas, suma_duraciones_criticas = calcular_actividades_criticas(grafo, duracion_actividades)
  # Guardar la salida en un archivo de texto
with open('/content/gdrive/My Drive/Resultado.txt', 'w') as f:
  # Redirigir la salida estándar al archivo
  import sys
  sys.stdout = f

  # Todo el código que imprime en la consola
  print(f"El tiempo mínimo para realizar el proyecto es de: {suma_duraciones_criticas} semanas")
  print("")
  print("Actividades críticas:")
  for actividad in actividades_criticas:
    num_actividad = actividad[1]
    descripcion = AE.loc[AE["Actividad"] == num_actividad, "Descripcion"].iloc[0]
    print(f"{num_actividad}: {descripcion}")

  # Restaurar la salida estándar
  sys.stdout = sys.__stdout__

main()